In [ ]:
#-----------------------------------------------------------------------------------------------------------
import matplotlib.pyplot as plt            # Plotting library
import cartopy, cartopy.crs as ccrs        # Plot maps
import cartopy.io.shapereader as shpreader # Import shapefiles
import numpy as np                         # Scientific computing with Python
import matplotlib                          # Comprehensive library for creating static, animated, and interactive visualizations in Python 
import xarray as xr                        # N-D labeled arrays and datasets in Python

#-------------------------------------------------------------------------------------------
# Abrir o arquivo NetCDF
data = xr.open_dataset('/home/arthursilva/projetos/Scripts/python/apnt/Samples/MONAN_DIAG_R_POS_GFS_2024100900_2024101008.mm.x1.40962L55.nc')
#data = data.sel(time='2015-07-26T18:00:00')

data = data.sel(longitude=slice(-67, -42), latitude=slice(-15, 10))
data

# Verificar a variável level e converter para hPa, se estiver em Pa
if 'level' in data.coords:
    # Converter para hPa
    data.coords['level'] = data.coords['level'] / 100
    data.coords['level'].attrs['units'] = 'hPa'

print(data)

ucomp = data['uvmt_isobaric'].sel(level=250)
vcomp = data['vvmt_isobaric'].sel(level=250)


In [ ]:
import matplotlib.pyplot as plt            # Plotting library
import cartopy, cartopy.crs as ccrs        # Plot maps
import cartopy.io.shapereader as shpreader # Import shapefiles
import numpy as np                         # Scientific computing with Python
import matplotlib                          # Comprehensive library for creating static, animated, and interactive visualizations in Python 
import xarray as xr                        # N-D labeled arrays and datasets in Python

#-------------------------------------------------------------------------------------------
# Abrir o arquivo NetCDF
data = xr.open_dataset('/home/arthursilva/projetos/Scripts/python/apnt/Samples/postprd/MONAN_DIAG_R_POS_GFS_2024100900_2024100901.mm.x1.40962L55.nc')
data

In [ ]:
#-----------------------------------------------------------------------------------------------------------
import matplotlib.pyplot as plt            # Plotting library
import cartopy, cartopy.crs as ccrs        # Plot maps
import cartopy.io.shapereader as shpreader # Import shapefiles
import numpy as np                         # Scientific computing with Python
import matplotlib                          # Comprehensive library for creating static, animated, and interactive visualizations in Python 
import xarray as xr                        # N-D labeled arrays and datasets in Python

#-------------------------------------------------------------------------------------------
# Abrir o arquivo NetCDF
data = xr.open_dataset('/home/arthursilva/projetos/Scripts/python/apnt/Samples/postprd/MONAN_DIAG_R_POS_GFS_2024101500_2024101607.mm.x1.40962L55.nc')

data = data.sel(longitude=slice(-63, -38), latitude=slice(-42, -17))

# Verificar a variável level e converter para hPa, se estiver em Pa
if 'level' in data.coords:
    data.coords['level'] = data.coords['level'] / 100
    data.coords['level'].attrs['units'] = 'hPa'

# Selecionar componentes do vento
ucomp = data['uvmt_isobaric'].sel(level=850).squeeze()  # Squeeze para remover dimensões extras
vcomp = data['vvmt_isobaric'].sel(level=850).squeeze()  # Squeeze para remover dimensões extras

# Calcular a velocidade do vento
ws = np.sqrt(ucomp ** 2 + vcomp ** 2)  # ws agora deve ser 2D

# Obter a data
time_var = data.Time.values
if np.isscalar(time_var):
    valid = str(time_var)
else:
    valid = str(time_var[0])
valid = valid.replace('-', '').replace('T', '').replace(':', '')[:10]
formatted_date = f"{valid[:4]}-{valid[4:6]}-{valid[6:8]}"
hora = f"{valid[8:10]}UTC"

# Escolher o tamanho do gráfico (largura x altura, em polegadas)
plt.figure(figsize=(13, 13))

# Criar um novo mapa
ax = plt.axes(projection=ccrs.PlateCarree())
img_extent = [data.longitude.min(), data.longitude.max(), data.latitude.min(), data.latitude.max()]

# Adicionar o shapefile
shapefile = list(shpreader.Reader('/home/arthursilva/projetos/Scripts/python/satelite/nwp-python-jul-2021/BR_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=0.9)

# Adicionar coastlines, borders e gridlines
ax.coastlines(resolution='10m', color='black', linewidth=0.8)
ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25,
                  xlocs=np.arange(-180, 180, 5), ylocs=np.arange(-90, 90, 5), draw_labels=True)
gl.top_labels = False
gl.right_labels = False

# Definir o intervalo dos contornos
data_min = 0
data_max = 60 
interval = 5
levels = np.arange(data_min, data_max, interval)

# Criar uma paleta de cores personalizada 
colors = ["#e7f2f4", "#ceeaee", "#b6e2e8", "#abdcff", "#a4d685", "#9cd04e", 
          "#abcf2a", "#c9d21b", "#e8d50c", "#ffd100", "#ffba00", "#ffa200"]
cmap = matplotlib.colors.ListedColormap(colors)
cmap.set_over('#ff8c00')
cmap.set_under('#fffafa')

# Plotar os contornos da velocidade do vento
contourf = ax.contourf(data.longitude, data.latitude, ws, transform=ccrs.PlateCarree(), levels=levels, cmap=cmap, extend='both', add_colorbar=False)
contour = ax.contour(data.longitude, data.latitude, ws, transform=ccrs.PlateCarree(), levels=levels, colors='black', linewidths=0.5)
ax.clabel(contour, inline=1, fontsize=10, fmt='%1.0f', colors='black')

# Plotar os vetores de vento (quiver plot)
# Diminua ainda mais a subamostragem para aproximar os vetores
#img3 = ax.quiver(data.longitude.values[::1], data.latitude.values[::1], 
#                 ucomp.values[::1, ::1], vcomp.values[::1, ::1], 
#                 transform=ccrs.PlateCarree(), scale=350, width=0.003, color='black')

# Adicionar a chave de quiver
#qk = ax.quiverkey(img3, 0.50, 0.89, 20, '20 m/s', labelpos='E', coordinates='figure')

# Plotar os vetores de vento como streamlines
stream = ax.streamplot(data.longitude.values, data.latitude.values, 
                       ucomp.values, vcomp.values, 
                       color='black', linewidth=1, density=1)

# Add a colorbar
plt.colorbar(contourf, label='Isotachs (kt)', orientation='vertical', pad=0.05, fraction=0.05)


plt.title('MONAN: Wind (250 hPa)' , fontweight='bold', fontsize=10, loc='left')
#plt.title('Valid: ' + formatted_date + hora, fontsize=10, loc='right')
plt.title('Valid: ' + formatted_date + ' ' + hora, fontsize=10, loc='right')

#plt.colorbar(contourf, ax=ax, orientation='vertical', label='Velocidade do Vento (m/s)', pad=0.02)
plt.show()


In [ ]:
import os
import matplotlib.pyplot as plt
import cartopy, cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import numpy as np
import matplotlib
import xarray as xr

# Diretório onde estão os arquivos NetCDF
diretorio_dados = '/home/arthursilva/projetos/Scripts/python/apnt/Samples/postprd/'

# Shapefile
shapefile_path = '/home/arthursilva/projetos/Scripts/python/satelite/nwp-python-jul-2021/BR_UF_2019.shp'

# Lista todos os arquivos .nc no diretório
arquivos_nc = sorted([f for f in os.listdir(diretorio_dados) if f.endswith('.nc')])

# Loop pelos arquivos .nc
for arquivo in arquivos_nc:
    # Abrir o arquivo NetCDF
    data = xr.open_dataset(os.path.join(diretorio_dados, arquivo))
    
    # Selecionar a área de interesse
    data = data.sel(longitude=slice(-63, -38), latitude=slice(-42, -17))

    
    # Verificar a variável level e converter para hPa, se estiver em Pa
    if 'level' in data.coords:
        data.coords['level'] = data.coords['level'] / 100
        data.coords['level'].attrs['units'] = 'hPa'

    # Selecionar componentes do vento
    ucomp = data['uvmt_isobaric'].sel(level=850).squeeze()
    vcomp = data['vvmt_isobaric'].sel(level=850).squeeze()

    # Calcular a velocidade do vento
    ws = np.sqrt(ucomp ** 2 + vcomp ** 2)

    # Obter a data
    time_var = data.Time.values
    if np.isscalar(time_var):
        valid = str(time_var)
    else:
        valid = str(time_var[0])
    valid = valid.replace('-', '').replace('T', '').replace(':', '')[:10]
    formatted_date = f"{valid[:4]}-{valid[4:6]}-{valid[6:8]}"
    hora = f"{valid[8:10]}UTC"

    # Criar gráfico
    plt.figure(figsize=(13, 13))
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    # Adicionar o shapefile
    shapefile = list(shpreader.Reader(shapefile_path).geometries())
    ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=0.9)
    
    # Adicionar coastlines, borders e gridlines
    ax.coastlines(resolution='10m', color='black', linewidth=0.8)
    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25,
                      xlocs=np.arange(-180, 180, 5), ylocs=np.arange(-90, 90, 5), draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False

    # Definir o intervalo dos contornos
    data_min = 0
    data_max = 60 
    interval = 5
    levels = np.arange(data_min, data_max, interval)

    # Criar uma paleta de cores personalizada 
    colors = ["#e7f2f4", "#ceeaee", "#b6e2e8", "#abdcff", "#a4d685", "#9cd04e", 
              "#abcf2a", "#c9d21b", "#e8d50c", "#ffd100", "#ffba00", "#ffa200"]
    cmap = matplotlib.colors.ListedColormap(colors)
    cmap.set_over('#ff8c00')
    cmap.set_under('#fffafa')

    # Plotar os contornos da velocidade do vento
    contourf = ax.contourf(data.longitude, data.latitude, ws, transform=ccrs.PlateCarree(), levels=levels, cmap=cmap, extend='both', add_colorbar=False)
    contour = ax.contour(data.longitude, data.latitude, ws, transform=ccrs.PlateCarree(), levels=levels, colors='black', linewidths=0.5)
    ax.clabel(contour, inline=1, fontsize=10, fmt='%1.0f', colors='black')

    # Plotar os vetores de vento (quiver plot)
    #img3 = ax.quiver(data.longitude.values[::1], data.latitude.values[::1], 
    #                 ucomp.values[::1, ::1], vcomp.values[::1, ::1], 
    #                 transform=ccrs.PlateCarree(), scale=250, width=0.003, color='black')

    # Adicionar a chave de quiver
    #qk = ax.quiverkey(img3, 0.50, 0.89, 20, '20 m/s', labelpos='E', coordinates='figure')

    stream = ax.streamplot(data.longitude.values, data.latitude.values, 
                       ucomp.values, vcomp.values, 
                       color='black', linewidth=1, density=1)

    # Adicionar a barra de cores
    plt.colorbar(contourf, label='Isotachs (kt)', orientation='vertical', pad=0.05, fraction=0.05)

    plt.title('MONAN: Wind (850 hPa)', fontweight='bold', fontsize=10, loc='left')
    plt.title('Valid: ' + formatted_date + ' ' + hora, fontsize=10, loc='right')

    # Salvar a figura
    #plt.savefig(os.path.join(diretorio_dados, f"vento_250hPa_{formatted_date}_{hora}.png"), dpi=300)
    
    plt.savefig(f'wind_plot-850_{formatted_date}_{hora}.png', bbox_inches='tight')

    # Fechar a figura atual
    plt.close()

print("Gráficos gerados e salvos com sucesso!")


In [ ]:
import os
import imageio
import matplotlib.pyplot as plt

# Defina o diretório onde estão as figuras salvas
diretorio_figuras = '/home/arthursilva/projetos/Scripts/python/MONAN/'  # Substitua pelo caminho real

# Obtenha a lista de todos os arquivos .png no diretório e ordene
arquivos_figuras = sorted([f for f in os.listdir(diretorio_figuras) if f.endswith('.png')])

# Defina o caminho para salvar a animação
caminho_animacao = '/home/arthursilva/projetos/Scripts/python/MONAN/animacao-vento-sul-850.gif'  # Substitua pelo caminho desejado

# Criar uma lista para armazenar as imagens
imagens = []

# Ler cada imagem e adicionar à lista
for arquivo in arquivos_figuras:
    caminho_arquivo = os.path.join(diretorio_figuras, arquivo)
    imagem = imageio.imread(caminho_arquivo)
    imagens.append(imagem)

# Criar a animação
imageio.mimsave(caminho_animacao, imagens, duration=80)  # A duração pode ser ajustada

print("Animação gerada com sucesso.")


Pressure + wind

In [ ]:
#-----------------------------------------------------------------------------------------------------------
import matplotlib.pyplot as plt            # Plotting library
import cartopy, cartopy.crs as ccrs        # Plot maps
import cartopy.io.shapereader as shpreader # Import shapefiles
import numpy as np                         # Scientific computing with Python
import matplotlib                          # Comprehensive library for creating static, animated, and interactive visualizations in Python 
import xarray as xr                        # N-D labeled arrays and datasets in Python

#-------------------------------------------------------------------------------------------
# Abrir o arquivo NetCDF
data = xr.open_dataset('/home/arthursilva/projetos/Scripts/python/apnt/Samples/postprd/MONAN_DIAG_R_POS_GFS_2024100900_2024100901.mm.x1.40962L55.nc')

data = data.sel(longitude=slice(-93, -69), latitude=slice(15, 40))

# Verificar a variável level e converter para hPa, se estiver em Pa
if 'level' in data.coords:
    data.coords['level'] = data.coords['level'] / 100
    data.coords['level'].attrs['units'] = 'hPa'

# Selecionar componentes do vento
ucomp = data['uvmt_isobaric'].sel(level=950).squeeze()  # Squeeze para remover dimensões extras
vcomp = data['vvmt_isobaric'].sel(level=950).squeeze()  # Squeeze para remover dimensões extras

# Calcular a pressão ao nível do mar
pmsl = data['mslp'] / 100  # Converter de Pa para hPa, se necessário

# Calculate the wind speed
ws = np.sqrt(ucomp ** 2 + vcomp ** 2)

# Obter a data
time_var = data.Time.values
if np.isscalar(time_var):
    valid = str(time_var)
else:
    valid = str(time_var[0])
valid = valid.replace('-', '').replace('T', '').replace(':', '')[:10]
formatted_date = f"{valid[:4]}-{valid[4:6]}-{valid[6:8]}"
hora = f"{valid[8:10]}UTC"

# Create a new map
plt.figure(figsize=(13, 13))
ax = plt.axes(projection=ccrs.PlateCarree())
img_extent = [data.longitude.min(), data.longitude.max(), data.latitude.min(), data.latitude.max()]

# Add the shapefile
shapefile = list(shpreader.Reader('/home/arthursilva/projetos/Scripts/python/satelite/nwp-python-jul-2021/BR_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=0.9)

# Adicionar coastlines, borders e gridlines
ax.coastlines(resolution='10m', color='black', linewidth=0.8)
ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 5), ylocs=np.arange(-90, 90, 5), draw_labels=True)
gl.top_labels = False
gl.right_labels = False

# Define o intervalo de contorno
data_min = 990
data_max = 1050
interval = 2
levels = np.arange(data_min, data_max, interval)

# Create a custom color palette 
colors = ["#310d00", "#631b00", "#942800", "#c53500", "#fd6123", "#fb824e", 
          "#faa679", "#f8c8a3", "#f6f6e1", "#f8f8e7", "#fafaee", "#fcfcf5", 
          "#fefefc", "#e8eef5", "#d9e2ef", "#c9d7e9", "#bacce2", "#a9bbd9", 
          "#95a1c9", "#8187b9", "#6d6da8", "#595398", "#463b87", "#382f6c", 
          "#2a2351", "#1c1836", "#0e161b"]
cmap = matplotlib.colors.ListedColormap(colors)
cmap.set_over('#000000')
cmap.set_under('#000000')


# Selecionar a primeira fatia de tempo
pmsl_slice = pmsl.isel(Time=0)  # Se você tem apenas um tempo, pode usar isel(Time=0)

# Plotar os contornos do vento
contourf = pmsl_slice.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels=levels, cmap=cmap, extend='both', add_colorbar=False)
contour = pmsl_slice.plot.contour(ax=ax, transform=ccrs.PlateCarree(), levels=levels, colors='black', linewidths=0.5)
ax.clabel(contour, inline=1, fontsize=10, fmt='%1.0f', colors='black')

# Create a flag to determine which barbs are flipped
flip_flag = np.zeros((ucomp.shape[0], ucomp.shape[1]))
                           
# All flags below the equator will be flipped
flip_flag[data.latitude.values < 0] = 1

# Plotar os vetores de vento (quiver plot)
#img3 = ax.barbs(data.longitude.values[::1], data.latitude.values[::1], 
#                 ucomp.values[::1, ::1], vcomp.values[::1, ::1], 
#                 length=5.0, linewidth=0.8, pivot='middle', barbcolor='gray', flagcolor='black', flip_barb=flip_flag[::1, ::1])

# Definir a escala do quiver
scale = 500  # Ajuste esse valor conforme necessário para uma visualização adequada

# Usar quiver ao invés de barbs
q = ax.quiver(data.longitude.values[::1], data.latitude.values[::1], 
               ucomp.values[::1, ::1], vcomp.values[::1, ::1], 
               scale=scale, color='black', pivot='middle')

# Adicionar uma legenda para o quiver (opcional)
#qk = ax.quiverkey(q, 0.9, 0.9, 10, '10 m/s', labelpos='E', coordinates='figure')


# Add a colorbar
cbar = plt.colorbar(contourf, ax=ax, orientation='vertical')
cbar.set_label('PSML (hPa)', fontweight='bold', fontsize=10)

# Ajustar o eixo y
ax.set_ylim(data.latitude.min(), data.latitude.max())

# Remove o colorbar da parte superior
plt.gca().set_title('', loc='center', fontsize=12, pad=20)
plt.gca().set_xlabel('')

# Add a title
plt.title('MONAN: PMSL & Winds (925 hPa)' , fontweight='bold', fontsize=10, loc='left')
plt.title('Valid: ' + formatted_date + ' ' + hora, fontsize=10, loc='right')

# Save the image
#plt.savefig('image_15.png')

# Show the image
plt.show()


In [ ]:
import matplotlib.pyplot as plt            # Plotting library
import cartopy, cartopy.crs as ccrs        # Plot maps
import cartopy.io.shapereader as shpreader # Import shapefiles
import numpy as np                         # Scientific computing with Python
import matplotlib                          # Comprehensive library for creating static, animated, and interactive visualizations in Python 
import xarray as xr                        # N-D labeled arrays and datasets in Python

#-------------------------------------------------------------------------------------------
# Abrir o arquivo NetCDF
data = xr.open_dataset('/home/arthursilva/projetos/Scripts/python/apnt/Samples/postprd/MONAN_DIAG_R_POS_GFS_2024101500_2024101607.mm.x1.40962L55.nc')

data = data.sel(longitude=slice(-63, -38), latitude=slice(-42, -17))

# Verificar a variável level e converter para hPa, se estiver em Pa
if 'level' in data.coords:
    data.coords['level'] = data.coords['level'] / 100
    data.coords['level'].attrs['units'] = 'hPa'

# Selecionar componentes do vento
ucomp = data['uvmt_isobaric'].sel(level=850).squeeze()  # Squeeze para remover dimensões extras
vcomp = data['vvmt_isobaric'].sel(level=850).squeeze()  # Squeeze para remover dimensões extras

# Calcular a pressão ao nível do mar
pmsl = data['mslp'] / 100  # Converter de Pa para hPa, se necessário

# Calculate the wind speed
ws = np.sqrt(ucomp ** 2 + vcomp ** 2)

# Obter a data
time_var = data.Time.values
if np.isscalar(time_var):
    valid = str(time_var)
else:
    valid = str(time_var[0])
valid = valid.replace('-', '').replace('T', '').replace(':', '')[:10]
formatted_date = f"{valid[:4]}-{valid[4:6]}-{valid[6:8]}"
hora = f"{valid[8:10]}UTC"

# Create a new map
plt.figure(figsize=(13, 13))
ax = plt.axes(projection=ccrs.PlateCarree())
img_extent = [data.longitude.min(), data.longitude.max(), data.latitude.min(), data.latitude.max()]

# Add the shapefile
shapefile = list(shpreader.Reader('/home/arthursilva/projetos/Scripts/python/satelite/nwp-python-jul-2021/BR_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=0.9)

# Adicionar coastlines, borders e gridlines
ax.coastlines(resolution='10m', color='black', linewidth=0.8)
ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 5), ylocs=np.arange(-90, 90, 5), draw_labels=True)
gl.top_labels = False
gl.right_labels = False

# Define o intervalo de contorno
data_min = 990
data_max = 1050
interval = 2
levels = np.arange(data_min, data_max, interval)

# Create a custom color palette 
colors = ["#310d00", "#631b00", "#942800", "#c53500", "#fd6123", "#fb824e", 
          "#faa679", "#f8c8a3", "#f6f6e1", "#f8f8e7", "#fafaee", "#fcfcf5", 
          "#fefefc", "#e8eef5", "#d9e2ef", "#c9d7e9", "#bacce2", "#a9bbd9", 
          "#95a1c9", "#8187b9", "#6d6da8", "#595398", "#463b87", "#382f6c", 
          "#2a2351", "#1c1836", "#0e161b"]
cmap = matplotlib.colors.ListedColormap(colors)
cmap.set_over('#000000')
cmap.set_under('#000000')

# Selecionar a primeira fatia de tempo
pmsl_slice = pmsl.isel(Time=0)  # Se você tem apenas um tempo, pode usar isel(Time=0)

# Plotar os contornos da pressão
contourf = pmsl_slice.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels=levels, cmap=cmap, extend='both', add_colorbar=False)
contour = pmsl_slice.plot.contour(ax=ax, transform=ccrs.PlateCarree(), levels=levels, colors='black', linewidths=0.5)
ax.clabel(contour, inline=1, fontsize=10, fmt='%1.0f', colors='black')

# Plotar os vetores de vento como streamlines
#stream = ax.streamplot(data.longitude.values, data.latitude.values,
#                        ucomp.values, vcomp.values,
#                        color='black', linewidth=0.5, density=1)

# Add a colorbar
cbar = plt.colorbar(contourf, ax=ax, orientation='vertical')
cbar.set_label('PMSL (hPa)', fontweight='bold', fontsize=10)

# Ajustar o eixo y
ax.set_ylim(data.latitude.min(), data.latitude.max())

# Remove o colorbar da parte superior
plt.gca().set_title('', loc='center', fontsize=12, pad=20)
plt.gca().set_xlabel('')

# Add a title
plt.title('MONAN: PMSL & Winds (850 hPa)', fontweight='bold', fontsize=10, loc='left')
plt.title('Valid: ' + formatted_date + ' ' + hora, fontsize=10, loc='right')

# Show the image
plt.show()


In [ ]:
import matplotlib.pyplot as plt            # Plotting library
import cartopy, cartopy.crs as ccrs        # Plot maps
import cartopy.io.shapereader as shpreader # Import shapefiles
import numpy as np                         # Scientific computing with Python
import matplotlib                          # Comprehensive library for creating static, animated, and interactive visualizations in Python 
import xarray as xr                        # N-D labeled arrays and datasets in Python

#-------------------------------------------------------------------------------------------
# Abrir o arquivo NetCDF
data = xr.open_dataset('/home/arthursilva/projetos/Scripts/python/apnt/Samples/postprd/MONAN_DIAG_R_POS_GFS_2024100900_2024101000.mm.x1.40962L55.nc')

# Recortar a área de interesse
data = data.sel(longitude=slice(-63, -38), latitude=slice(-42, -17))

# Verificar a variável level e converter para hPa, se estiver em Pa
if 'level' in data.coords:
    data.coords['level'] = data.coords['level'] / 100
    data.coords['level'].attrs['units'] = 'hPa'

# Selecionar componentes do vento
ucomp = data['uvmt_isobaric'].sel(level=950).squeeze()  # Squeeze para remover dimensões extras
vcomp = data['vvmt_isobaric'].sel(level=950).squeeze()  # Squeeze para remover dimensões extras

# Selecionar a variável de precipitação
prec = data['prec'].isel(Time=0)  # Selecione o primeiro índice de tempo, caso haja mais de um

# Obter a data
time_var = data.Time.values
if np.isscalar(time_var):
    valid = str(time_var)
else:
    valid = str(time_var[0])
valid = valid.replace('-', '').replace('T', '').replace(':', '')[:10]
formatted_date = f"{valid[:4]}-{valid[4:6]}-{valid[6:8]}"
hora = f"{valid[8:10]}UTC"

# Create a new map
plt.figure(figsize=(13, 13))
ax = plt.axes(projection=ccrs.PlateCarree())
img_extent = [data.longitude.min(), data.longitude.max(), data.latitude.min(), data.latitude.max()]

# Add the shapefile
shapefile = list(shpreader.Reader('/home/arthursilva/projetos/Scripts/python/satelite/nwp-python-jul-2021/BR_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=0.9)

# Adicionar coastlines, borders e gridlines
ax.coastlines(resolution='10m', color='black', linewidth=0.8)
ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 5), ylocs=np.arange(-90, 90, 5), draw_labels=True)
gl.top_labels = False
gl.right_labels = False

# Define o intervalo de contorno para precipitação
prec_min = 0.1
prec_max = 18  # Ajuste conforme necessário para o intervalo de precipitação
interval = 1
levels = np.arange(prec_min, prec_max, interval)

# Criar um esquema de cores customizado para precipitação
cmap = plt.cm.Blues

# Plotar a precipitação
contourf = prec.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels=levels, cmap=cmap, extend='both', add_colorbar=False)

# Adicionar o vetor de vento como streamlines
stream = ax.streamplot(data.longitude.values, data.latitude.values,
                        ucomp.values, vcomp.values,
                        color='black', linewidth=1, density=1)

# Adicionar a colorbar
cbar = plt.colorbar(contourf, ax=ax, orientation='vertical')
cbar.set_label('Precipitação (mm)', fontweight='bold', fontsize=10)

# Ajustar o eixo y
ax.set_ylim(data.latitude.min(), data.latitude.max())

# Adicionar título
plt.title('MONAN: Precipitação & Ventos (950 hPa)', fontweight='bold', fontsize=10, loc='left')
plt.title('Valid: ' + formatted_date + ' ' + hora, fontsize=10, loc='right')

# Exibir a imagem
plt.show()


In [ ]:
data = xr.open_dataset('/home/arthursilva/projetos/Scripts/python/apnt/Samples/postprd/MONAN_DIAG_R_POS_GFS_2024101500_2024101607.mm.x1.40962L55.nc')
data

In [ ]:
import matplotlib.pyplot as plt            # Plotting library
import cartopy, cartopy.crs as ccrs        # Plot maps
import cartopy.io.shapereader as shpreader # Import shapefiles
import numpy as np                         # Scientific computing with Python
import matplotlib                          # Comprehensive library for creating static, animated, and interactive visualizations in Python 
import xarray as xr                        # N-D labeled arrays and datasets in Python

#-------------------------------------------------------------------------------------------
# Abrir o arquivo NetCDF
data = xr.open_dataset('/home/arthursilva/projetos/Scripts/python/apnt/Samples/postprd/MONAN_DIAG_R_POS_GFS_2024101500_2024101607.mm.x1.40962L55.nc')

# Recortar a área de interesse
#data = data.sel(longitude=slice(-93, -69), latitude=slice(15, 40))

# Recortar região sul do Brasil
data = data.sel(longitude=slice(-63, -38), latitude=slice(-42, -17))

# Verificar a variável level e converter para hPa, se estiver em Pa
if 'level' in data.coords:
    data.coords['level'] = data.coords['level'] / 100
    data.coords['level'].attrs['units'] = 'hPa'

# Selecionar componentes do vento
ucomp = data['uvmt_isobaric'].sel(level=850).squeeze()  # Squeeze para remover dimensões extras
vcomp = data['vvmt_isobaric'].sel(level=850).squeeze()  # Squeeze para remover dimensões extras

# Selecionar a variável de precipitação
prec = data['prec'].isel(Time=0)  # Selecione o primeiro índice de tempo, caso haja mais de um

# Obter a data
time_var = data.Time.values
if np.isscalar(time_var):
    valid = str(time_var)
else:
    valid = str(time_var[0])
valid = valid.replace('-', '').replace('T', '').replace(':', '')[:10]
formatted_date = f"{valid[:4]}-{valid[4:6]}-{valid[6:8]}"
hora = f"{valid[8:10]}UTC"

# Criar o mapa
plt.figure(figsize=(13, 13))
ax = plt.axes(projection=ccrs.PlateCarree())
img_extent = [data.longitude.min(), data.longitude.max(), data.latitude.min(), data.latitude.max()]

# Add shapefile
shapefile = list(shpreader.Reader('/home/arthursilva/projetos/Scripts/python/satelite/nwp-python-jul-2021/BR_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=0.9)

# Adicionar coastlines, borders e gridlines
ax.coastlines(resolution='10m', color='black', linewidth=0.8)
ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 5), ylocs=np.arange(-90, 90, 5), draw_labels=True)
gl.top_labels = False
gl.right_labels = False

# Define o intervalo de contorno para precipitação
prec_min = 0.1
prec_max = 18  # Ajuste conforme necessário para o intervalo de precipitação
interval = 1
levels = np.arange(prec_min, prec_max, interval)

# Criar uma paleta de cores personalizada
colors = ["#b4f0f0", "#96d2fa", "#78b9fa", "#3c95f5", "#1e6deb", "#1463d2", 
          "#0fa00f", "#28be28", "#50f050", "#72f06e", "#b3faaa", "#fff9aa", 
          "#ffe978", "#ffc13c", "#ffa200", "#ff6200", "#ff3300", "#ff1500", 
          "#c00100", "#a50200", "#870000", "#653b32"]
cmap = matplotlib.colors.ListedColormap(colors)
cmap.set_over('#000000')
cmap.set_under('#ffffff')

# Plotar a precipitação
contourf = prec.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels=levels, cmap=cmap, extend='both', add_colorbar=False)

# Adicionar o vetor de vento como streamlines
stream = ax.streamplot(data.longitude.values, data.latitude.values,
                        ucomp.values, vcomp.values,
                        color='black', linewidth=1, density=1)

# Adicionar a colorbar
cbar = plt.colorbar(contourf, ax=ax, orientation='vertical', shrink=0.8)
cbar.set_label('Precipitação (mm)', fontweight='bold', fontsize=10)

# Ajustar o eixo y
ax.set_ylim(data.latitude.min(), data.latitude.max())

# Remove o colorbar da parte superior
plt.gca().set_title('', loc='center', fontsize=12, pad=20)
plt.gca().set_xlabel('')

# Adicionar título
plt.title('MONAN: Precipitação & Ventos (850 hPa)', fontweight='bold', fontsize=10, loc='left')
plt.title('Valid: ' + formatted_date + ' ' + hora, fontsize=10, loc='right')

# Exibir a imagem
plt.show()


In [ ]:
import os
import matplotlib.pyplot as plt
import cartopy, cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import numpy as np
import matplotlib
import xarray as xr

# Caminho para os arquivos NetCDF
data_folder = '/home/arthursilva/projetos/Scripts/python/apnt/Samples/postprd/'
output_folder = '/home/arthursilva/projetos/Scripts/python/apnt/Samples/figures/'  # Defina o caminho para salvar as figuras

# Certifique-se de que a pasta de saída existe
os.makedirs(output_folder, exist_ok=True)

# Listar todos os arquivos .nc e ordenar
nc_files = sorted([f for f in os.listdir(data_folder) if f.endswith('.nc')])

for nc_file in nc_files:
    # Abrir o arquivo NetCDF
    data = xr.open_dataset(os.path.join(data_folder, nc_file))

    # Selecionar a área desejada
    data = data.sel(longitude=slice(-63, -38), latitude=slice(-42, -17))

    # Verificar a variável level e converter para hPa, se necessário
    if 'level' in data.coords:
        data.coords['level'] = data.coords['level'] / 100
        data.coords['level'].attrs['units'] = 'hPa'

    # Selecionar componentes do vento
    ucomp = data['uvmt_isobaric'].sel(level=950).squeeze()
    vcomp = data['vvmt_isobaric'].sel(level=950).squeeze()

    # Calcular a pressão ao nível do mar
    pmsl = data['mslp'] / 100

    # Calcular a velocidade do vento
    ws = np.sqrt(ucomp ** 2 + vcomp ** 2)

    # Obter a data
    time_var = data.Time.values
    if np.isscalar(time_var):
        valid = str(time_var)
    else:
        valid = str(time_var[0])
    valid = valid.replace('-', '').replace('T', '').replace(':', '')[:10]
    formatted_date = f"{valid[:4]}-{valid[4:6]}-{valid[6:8]}"
    hora = f"{valid[8:10]}UTC"

    # Criar o mapa
    plt.figure(figsize=(13, 13))
    ax = plt.axes(projection=ccrs.PlateCarree())
    img_extent = [data.longitude.min(), data.longitude.max(), data.latitude.min(), data.latitude.max()]

    # Adicionar o shapefile
    shapefile = list(shpreader.Reader('/home/arthursilva/projetos/Scripts/python/satelite/nwp-python-jul-2021/BR_UF_2019.shp').geometries())
    ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='black', facecolor='none', linewidth=0.9)

    # Adicionar coastlines, borders e gridlines
    ax.coastlines(resolution='10m', color='black', linewidth=0.8)
    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, 
                       xlocs=np.arange(-180, 180, 5), ylocs=np.arange(-90, 90, 5), draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False

    # Definir o intervalo de contorno
    data_min = 990
    data_max = 1050
    interval = 2
    levels = np.arange(data_min, data_max, interval)

    # Criar uma paleta de cores personalizada
    colors = ["#310d00", "#631b00", "#942800", "#c53500", "#fd6123", "#fb824e", 
              "#faa679", "#f8c8a3", "#f6f6e1", "#f8f8e7", "#fafaee", "#fcfcf5", 
              "#fefefc", "#e8eef5", "#d9e2ef", "#c9d7e9", "#bacce2", "#a9bbd9", 
              "#95a1c9", "#8187b9", "#6d6da8", "#595398", "#463b87", "#382f6c", 
              "#2a2351", "#1c1836", "#0e161b"]
    cmap = matplotlib.colors.ListedColormap(colors)
    cmap.set_over('#000000')
    cmap.set_under('#000000')

    # Selecionar a primeira fatia de tempo
    pmsl_slice = pmsl.isel(Time=0)

    # Plotar os contornos da pressão
    contourf = pmsl_slice.plot.contourf(ax=ax, transform=ccrs.PlateCarree(), levels=levels, cmap=cmap, extend='both', add_colorbar=False)
    contour = pmsl_slice.plot.contour(ax=ax, transform=ccrs.PlateCarree(), levels=levels, colors='black', linewidths=0.5)
    ax.clabel(contour, inline=1, fontsize=10, fmt='%1.0f', colors='black')

    # Plotar os vetores de vento como streamlines
    #stream = ax.streamplot(data.longitude.values, data.latitude.values,
    #                        ucomp.values, vcomp.values,
    #                        color='black', linewidth=1, density=1)

    # Adicionar um colorbar
    cbar = plt.colorbar(contourf, ax=ax, orientation='vertical')
    cbar.set_label('PMSL (hPa)', fontweight='bold', fontsize=10)

    # Ajustar o eixo y
    ax.set_ylim(data.latitude.min(), data.latitude.max())

    # Remover o colorbar da parte superior
    plt.gca().set_title('', loc='center', fontsize=12, pad=20)
    plt.gca().set_xlabel('')

    # Adicionar um título
    plt.title('MONAN: PMSL & Winds (950 hPa)', fontweight='bold', fontsize=10, loc='left')
    plt.title('Valid: ' + formatted_date + ' ' + hora, fontsize=10, loc='right')

    # Salvar a figura
    output_file = os.path.join(output_folder, f'pmsl_winds_{formatted_date}_{hora}.png')
    plt.savefig(output_file, bbox_inches='tight', dpi=300)

    # Fechar a figura para liberar memória
    plt.close()


In [ ]:
import os
import imageio
import matplotlib.pyplot as plt

# Defina o diretório onde estão as figuras salvas
diretorio_figuras = '/home/arthursilva/projetos/Scripts/python/apnt/Samples/figures/'  # Substitua pelo caminho real

# Obtenha a lista de todos os arquivos .png no diretório e ordene
arquivos_figuras = sorted([f for f in os.listdir(diretorio_figuras) if f.endswith('.png')])

# Defina o caminho para salvar a animação
caminho_animacao = '/home/arthursilva/projetos/Scripts/python/MONAN/animacao-pmsl.gif'  # Substitua pelo caminho desejado

# Criar uma lista para armazenar as imagens
imagens = []

# Ler cada imagem e adicionar à lista
for arquivo in arquivos_figuras:
    caminho_arquivo = os.path.join(diretorio_figuras, arquivo)
    imagem = imageio.imread(caminho_arquivo)
    imagens.append(imagem)

# Criar a animação
imageio.mimsave(caminho_animacao, imagens, duration=300)  # A duração pode ser ajustada

print("Animação gerada com sucesso.")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import tqdm
from matplotlib.animation import FuncAnimation
from IPython.display import Image
import xarray as xr

# Carregar dados e definir a região
data = xr.open_dataset('/home/arthursilva/projetos/Scripts/python/apnt/Samples/postprd/MONAN_DIAG_R_POS_GFS_2024100900_2024100901.mm.x1.40962L55.nc')
data = data.assign_coords(longitude=(((data.longitude + 180) % 360) - 180))
data = data.sel(longitude=slice(-93, -69), latitude=slice(15, 40))
data.coords['level'] = data.coords['level'] / 100
ucomp = data['uvmt_isobaric'].sel(level=250).squeeze()
vcomp = data['vvmt_isobaric'].sel(level=250).squeeze()
ws = np.sqrt(ucomp**2 + vcomp**2)
lons, lats = ucomp.longitude.values, ucomp.latitude.values
Y, X = np.meshgrid(lats, lons)
U, V = ucomp.values, vcomp.values
speed = ws.values

# Configuração de figura e níveis de contorno
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
img_extent = [-93, -69, 15, 40]
data_min, data_max, interval = 0, 60, 5
levels = np.arange(data_min, data_max, interval)
colors = ["#e7f2f4", "#ceeaee", "#b6e2e8", "#abdcff", "#a4d685", "#9cd04e"]
cf = ax.contourf(X, Y, speed, levels=levels, colors=colors, extend='both')
cbar = plt.colorbar(cf, ax=ax, orientation='horizontal', pad=0.05)
cbar.set_label('Velocidade do Vento (m/s)')
plt.title('GFS: Velocidade e Direção do Vento (250 hPa) - Animação', fontweight='bold', fontsize=10, loc='left')
plt.title('Reg.: ' + str(img_extent), fontsize=10, loc='right')

# Função de atualização para a animação das linhas de corrente
def update(frame_no):
    ax.clear()  # Limpa o plot para redesenhar as linhas de corrente
    # Atualizar os dados para cada frame (exemplo simples de modificação)
    phase_shift = 0.1 * frame_no  # Exemplo de deslocamento de fase
    U_frame = U * np.cos(phase_shift)  # Alterar U
    V_frame = V * np.sin(phase_shift)  # Alterar V
    speed_frame = np.sqrt(U_frame**2 + V_frame**2)  # Calcular nova velocidade

    ax.contourf(X, Y, speed_frame, levels=levels, colors=colors, extend='both')
    ax.streamplot(X, Y, U_frame, V_frame, color=speed_frame, linewidth=0.5, density=1.5, cmap='plasma')
    plt.title('GFS: Velocidade e Direção do Vento (250 hPa) - Animação', fontweight='bold', fontsize=10, loc='left')
    plt.title('Reg.: ' + str(img_extent), fontsize=10, loc='right')
    pbar.update()  # Atualiza a barra de progresso

# Criar a animação
n_frames = 30
pbar = tqdm.tqdm(total=n_frames)
animation = FuncAnimation(fig, update, frames=n_frames, interval=500)
animation.save('wind.gif', writer='imagemagick', fps=15)
pbar.close()

# Exibir a animação
# Display the GIF if needed
# Image(open('wind.gif', 'rb').read())
